# sbPoynETNRPy: An Einstein Toolkit Thorn for Computing the 4-Velocity Time-Component $u^0$, the Magnetic Field Measured by a Comoving Observer $b^{\mu}$, and the Poynting Vector $S^i$

<font color='red'>**This module has not yet been validated against a trusted code.**</font>

In the [previous tutorial module](Tutorial-u0_smallb_Poynting-Cartesian.ipynb), we constructed within SymPy full expressions for the 4-velocity time-component $u^0$, the magnetic field (measured by a comoving observer) $b^{\mu}$, and the Poynting vector $S^i$.

Here we will work through the steps necessary to construct an Einstein Toolkit diagnostic thorn (module) that uses ADMBase and HydroBase variables as input into the NRPy+-generated SymPy expressions for $b^{\mu}$, $b^2$, and the Poynting Vector $S^i$, outputting to gridfunctions $\text{smallb4U[]}$, $\text{smallb2}$, and $\text{SPoyn[]}$, respectively. This tutorial is in two steps:

1. Call on NRPy+ to convert the SymPy expressions for $b^{\mu}$, $b^2$, and the Poynting Vector $S^i$
1. Write the C code and build up the needed Einstein Toolkit infrastructure (i.e., the .ccl files).

In [1]:
# Step 1: import all needed modules from NRPy+:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *
import sympy as sp

In [2]:
# Step 2: Initialize  parameters (stub; there are none for this module)
thismodule = __name__

Additionally, we need to turn off verbose output in the outputC functions. This is an important step in this case, because our final expressions are very large. Verbose output, when enabled, will print (in comments) the input SymPy expressions to the top of the file $\it{without}$ CSE, resulting here in an *enormous* output file.

We will also declare the additional gridfunctions we need for this thorn:

**Inputs from ADMBase:**
* the physical metric $\gamma_{ij}$
* the spacetime gauge quantities $\alpha$ and $\beta^i$

**Inputs from HydroBase:**
* the Valencia 3-velocity $v^i_{(n)}$
* the densitized magnetic field of a normal observer $\tilde{B}^i$

**Output gridfunctions:**
* the magnetic field as observed in a frame comoving with the plasma $b^\mu$ ($\text{smallb4U[]}$)
* twice the magnetic pressure $2 P_{\rm mag} = b_\mu b^\mu = b^2$ ($\text{smallb2}$)
* the Poynting vector $S^i$ ($\text{SPoyn[]}$)

In [3]:
# Step 1a: Force outputC::outCverbose to be false for this module to avoid gigantic C files
#          filled with the non-CSE expressions for the Weyl scalars.
par.set_parval_from_str("outputC::outCverbose",False) # To prevent absurdly large output files.
# Step 1b: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Step 1c: declare the additional gridfunctions (i.e., functions whose values are declared 
#          at every grid point, either inside or outside of our SymPy expressions) needed
#         for this thorn:
#           * the physical metric $\gamma_{ij}$, 
#           * the extrinsic curvature $K_{ij}$, 
#           * the real and imaginary components of $\psi_4$, and
#           * the Weyl curvature invariants:

# INPUT GRIDFUNCTIONS:
gammaDD = ixp.register_gridfunctions_for_single_rank2("AUX","gammaDD", "sym01") # The AUX or EVOL designation is *not*
                                                                                # used in diagnostic modules.
beta = ixp.register_gridfunctions_for_single_rank1("AUX","betaU") # The AUX or EVOL designation is *not*
                                                                  # used in diagnostic modules.
alpha = gri.register_gridfunctions("AUX","alpha") # The AUX or EVOL designation is *not*
                                                  # used in diagnostic modules.
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUX","ValenciavU") # The AUX or EVOL designation is *not*
                                                                             # used in diagnostic modules.
BtildeU = ixp.register_gridfunctions_for_single_rank1("AUX","BtildeU") # The AUX or EVOL designation is *not*
                                                                       # used in diagnostic modules.

# OUTPUT GRIDFUNCTIONS:
smallb4U = ixp.register_gridfunctions_for_single_rank1("AUX","smallb4U",DIM=4) # The AUX or EVOL designation is *not*
                                                                               # used in diagnostic modules.
smallb2 = gri.register_gridfunctions("AUX","smallb2") # The AUX or EVOL designation is *not*
                                                    # used in diagnostic modules.
PoynSU = ixp.register_gridfunctions_for_single_rank1("AUX","PoynSU") # The AUX or EVOL designation is *not*
                                                                     # used in diagnostic modules.